In [13]:
!cat scheme.py

import math
import operator
from collections import namedtuple
import sys

### Type Definitions
Symbol = str              # A Scheme Symbol is implemented as a Python str
Number = (int, float)     # A Scheme Number is implemented as a Python int or float
Atom   = (Symbol, Number) # A Scheme Atom is a Symbol or Number
List   = list             # A Scheme List is implemented as a Python list
Exp    = (Atom, List)     # A Scheme expression is an Atom or List
# A Scheme environment is a mapping of {variable: value}, an environment may refer to a parent environment
Env    = namedtuple('Env', ['table', 'outer']) 
# A Scheme procedure has parameter, and body, and it is defined in an environment
Proc = namedtuple('Proc', ['parms', 'body', 'env'])

### Parsing
def tokenize(chars: str) -> list:
  """Converts a string of characters into a list of tokens."""
  return chars.replace('(', ' ( ').replace(')', ' ) ').split()

def parse(program: str) -> Exp:
  """Reads Scheme program from a string"""
  

In [1]:
from scheme import evaluate

In [2]:
p0 = '(print 1 2 3 "Hello")'
evaluate(p0)

1 2 3 Hello


In [3]:
p1 = "(begin (define r 10) (* pi (* r r)))"
evaluate(p1)

314.1592653589793

In [4]:
p2 = ("(car (list 1 2 3 4))")
evaluate(p2)

1

In [5]:
p3 = """
(begin 
  (define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1))))))
  (fact 100)
)
"""
%timeit evaluate(p3)
evaluate(p3)

862 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [6]:
fact = lambda n: 1 if n<=1 else n * fact(n-1)
%timeit fact(100)
fact(100)

12.1 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [7]:
p4 = """
(begin 
  (define twice (lambda (x) (* 2 x)))
  (twice 10)
)
"""
evaluate(p4)

20

In [8]:
p5 = """
(begin
  (define fib (lambda (n) (if (< n 2) 1 (+ (fib (- n 1)) (fib (- n 2))))))
  (fib 10)
)
"""
%timeit evaluate(p5)
evaluate(p5)

1.23 ms ± 7.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


89

In [9]:
fib = lambda n: 1 if n < 2 else fib(n - 1) + fib (n - 2)
%timeit fib(10)
fib(10)

13.6 µs ± 43.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


89

In [10]:
p6 = """
(begin
  (define move (lambda (n from to spare)
    (if 
      (= 0 n) "Done_Moving"
      (begin
        (move (- n 1) from spare to)
        (print "Move" n "from" from "to" to)
        (move (- n 1) spare to from)
      )
    ))
)
  (move 3 "A" "B" "C")
)
"""

evaluate(p6)

Move 1 from A to B
Move 2 from A to C
Move 1 from B to C
Move 3 from A to B
Move 1 from C to A
Move 2 from C to B
Move 1 from A to B


'Done_Moving'

In [11]:
p7 ="""
(begin
  (define threshold 1e-6)
  (define dx 1e-3)
  
  (define newton (lambda (f guess)
    (fixed-point (lambda (x) (- x (/ (f x) ((derive f threshold) x)))) guess)))

  (define derive (lambda (f dx)
    (lambda (x) (/ (- (f (+ x dx)) (f x)) dx))))

  (define close-enough? (lambda (x y)
    (< (abs (- x y)) threshold)))
    
  (define fixed-point (lambda (f n)
    (if (close-enough? n (f n))
        n
        (fixed-point f (f n)))))

  (define jazr (lambda (y)
    (newton (lambda (x) (- y (* x x)))  1.0)))

  (jazr 2)
)
"""
%timeit evaluate(p7)
evaluate(p7)

562 µs ± 4.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


1.4142135623754424

In [12]:
p8 = """
(begin
  (define deriv (lambda (exp var)
    (if (constant? exp var) 0
        (if  (same-var? exp var) 1
             (if (sum? exp) (make-sum (deriv (a1 exp) var) (deriv (a2 exp) var))
                 (if (product? exp) (make-sum
                           (make-product (m1 exp) (deriv (m2 exp) var))
                           (make-product (m2 exp) (deriv (m1 exp) var)))
                     "ERROR"))))))
         
  (define atomic? (lambda (exp) (not (list? exp))))

  (define constant? (lambda (exp var) (and (atomic? exp) (not (equal? exp var)))))

  (define same-var? (lambda (exp var) (and (atomic? exp) (equal? exp var))))

  (define sum? (lambda (exp) (and (list? exp) (equal? (car exp) (quote +)))))

  (define product? (lambda (exp) (and (list? exp) (equal? (car exp) (quote *)))))

  (define make-sum (lambda (a1 a2)
    (if (equal? a1 0) a2
        (if (equal? a2 0) a1
            (list (quote +) a1 a2)))))
  
  (define make-product (lambda (m1 m2)
    (if (equal? m1 1) m2
        (if (equal? m2 1) m1
            (if (or (equal? m1 0) (equal? m2 0)) 0
                (list (quote *) m1 m2))))))

  (define a1 (lambda (l) (car (cdr l))))

  (define a2 (lambda (l) (car (cdr (cdr l)))))

  (define m1 a1)

  (define m2 a2)

  (deriv (quote (+ (* x x) (* 2 x))) (quote x))
)
"""
%timeit evaluate(p8)
evaluate(p8)

753 µs ± 922 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


['+', ['+', 'x', 'x'], 2]